In [117]:
import pandas as pd
import numpy as np

import DataRetriever as dr

retriever = dr.DataRetriever()

df = retriever.get_data("All-Subsystems-minute-Year2.pkl")

pd.options.mode.chained_assignment = None

In [118]:
# Creating a new column with the time delta in seconds.
df["Timestamp_Delta"] = (df["Timestamp"] - df["Timestamp"].shift()).astype('timedelta64[s]')

In [119]:
df["Timestamp_Delta"].fillna(0, inplace=True)

In [120]:
df_index_split = df[df["Timestamp_Delta"] > 300].index

In [121]:
for row in df.index:
    dt = df["Timestamp"][row]
    df.at[row, "Timestamp"] = pd.Timestamp(year=dt.year, month=dt.month, day=dt.day,
                                       hour=dt.hour, minute=dt.minute, second=dt.second)

df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Interpolate measurements

In [122]:
df[df["HVAC_HeatPumpIndoorUnitPower"].isnull() == True]

,Timestamp,TimeStamp_Count,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,...,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow,Timestamp_Delta
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-20 01:03:37,2015-02-20 01:03:37,27385,0.055482,4.712000,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2015-02-20 01:04:37,2015-02-20 01:04:37,27386,0.058124,4.776172,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2015-02-20 01:05:37,2015-02-20 01:05:37,27387,0.058124,4.860834,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2015-02-20 01:06:37,2015-02-20 01:06:37,27388,0.058124,4.862597,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2015-02-20 01:07:37,2015-02-20 01:07:37,27389,0.058124,4.887749,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-30 01:06:29,2016-01-30 01:06:29,516794,0.070674,5.211040,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.018551,0.001090,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2016-01-30 01:07:29,2016-01-30 01:07:29,516795,0.071334,5.313943,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.017023,-0.000391,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2016-01-30 01:08:29,2016-01-30 01:08:29,516796,0.074636,5.391418,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.017164,-0.000776,NaN,NaN,NaN,NaN,NaN,NaN,60.0


In [123]:
df.isnull().sum()

Timestamp                          0
TimeStamp_Count                    0
Load_LatentHeatWaterVolume         0
Load_RefrigeratorTemp              0
Load_StatusBA1Lights               0
                                ... 
HVAC_DehumidifierPower           777
HVAC_DehumidifierInletAirTemp    777
HVAC_DehumidifierExitAirTemp     777
HVAC_DehumidifierAirflow         777
Timestamp_Delta                    0
Length: 383, dtype: int64

# Function to create interpolated records based on a DataFrame

In [124]:
df

,Timestamp,TimeStamp_Count,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,...,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow,Timestamp_Delta
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:01:50,2015-02-01 00:01:50,1,0.000000,4.994715,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,184.412,1108.765,4.471,72.102,74.655,0.0,0.0
2015-02-01 00:02:50,2015-02-01 00:02:50,2,0.000000,5.046792,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,184.618,1097.577,4.508,72.096,74.701,0.0,60.0
2015-02-01 00:03:50,2015-02-01 00:03:50,3,0.000000,5.129493,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,169.844,1086.113,4.510,72.077,74.759,0.0,60.0
2015-02-01 00:04:50,2015-02-01 00:04:50,4,0.001981,5.163539,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,167.300,1079.586,4.529,72.074,74.829,0.0,60.0
2015-02-01 00:05:50,2015-02-01 00:05:50,5,0.001981,5.232744,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,182.976,1048.502,4.438,72.086,74.863,0.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-31 23:54:24,2016-01-31 23:54:24,519600,1.757590,4.512704,0.0,0.0,0.0,0.0,0.0,0.0,...,942.630328,0.001164,0.000296,11.725,22.563,4.265,67.645,68.550,0.0,60.0
2016-01-31 23:55:24,2016-01-31 23:55:24,519601,1.760893,4.514255,0.0,0.0,0.0,0.0,0.0,0.0,...,942.630328,0.001546,0.000268,11.178,22.475,4.286,67.655,68.551,0.0,60.0
2016-01-31 23:56:24,2016-01-31 23:56:24,519602,1.763535,4.548853,0.0,0.0,0.0,0.0,0.0,0.0,...,942.630328,0.000187,-0.000285,10.767,22.388,4.293,67.644,68.549,0.0,60.0


In [135]:
def interpolate_df(dataframe):
    dataframe["DF"] = 1
    dataframe.drop(["Timestamp", "TimeStamp_Count", "Timestamp_Delta"], axis=1, inplace=True)

    interpolation_records = dataframe.resample('1T') # Rækker svarende til 1 min i forskel
    interpolation_records = interpolation_records.interpolate()
    interpolation_records["DF"] = 0

    df_concat = pd.concat([dataframe, interpolation_records], axis=0)
    df_concat.sort_index(inplace=True)

    df_concat.interpolate(method="time", inplace=True)

    df_concat = df_concat[1:]

    df_concat = df_concat[df_concat["DF"] == 0]

    del df_concat["DF"]

    df_concat["Timestamp"] = df_concat.index

    return df_concat

# Handle boolean attributes

In [126]:
metadata = retriever.get_data("metadata-year2.pkl")
metadata.rename(columns={"Unnamed: 0": "Attribute"}, inplace=True)

In [127]:
boolean_attributes = metadata["Attribute"][metadata["Units"] == "Binary Status"].tolist()

In [128]:
list_boolean_attributes = list(set(boolean_attributes) & set(df.columns.tolist())) # Some columns does not exist in df

In [129]:
def resolve_boolean(dataframe, columns=list_boolean_attributes):
    dataframe[columns] = dataframe[columns].round()
    return dataframe

# Sub-DataFrames interpolated

In [136]:
df_1_interpolated = resolve_boolean(dataframe=interpolate_df(dataframe=df[:df_index_split[0]]))
df_2_interpolated = resolve_boolean(dataframe=interpolate_df(dataframe=df[df_index_split[0]:df_index_split[1]]))
df_3_interpolated = resolve_boolean(dataframe=interpolate_df(dataframe=df[df_index_split[1]:df_index_split[2]]))
df_4_interpolated = resolve_boolean(dataframe=interpolate_df(dataframe=df[df_index_split[2]:]))

In [137]:
df_1_interpolated

,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,Load_StatusBR4Lights,...,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow,Timestamp
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:02:00,0.000000,5.003394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,184.446333,1106.900333,4.477167,72.101000,74.662667,0.0,2015-02-01 00:02:00
2015-02-01 00:03:00,0.000000,5.060575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,182.155667,1095.666333,4.508333,72.092833,74.710667,0.0,2015-02-01 00:03:00
2015-02-01 00:04:00,0.000330,5.135167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,169.420000,1085.025167,4.513167,72.076500,74.770667,0.0,2015-02-01 00:04:00
2015-02-01 00:05:00,0.001981,5.175073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,169.912667,1074.405333,4.513833,72.076000,74.834667,0.0,2015-02-01 00:05:00
2015-02-01 00:06:00,0.001981,5.252252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,181.102000,1049.749500,4.458500,72.084500,74.880000,0.0,2015-02-01 00:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-08 02:56:00,1.247685,5.226888,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,130.119000,1071.628000,4.424000,69.729000,71.949000,0.0,2015-03-08 02:56:00
2015-03-08 02:57:00,1.247685,5.226888,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,130.119000,1071.628000,4.424000,69.729000,71.949000,0.0,2015-03-08 02:57:00
2015-03-08 02:58:00,1.247685,5.226888,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,130.119000,1071.628000,4.424000,69.729000,71.949000,0.0,2015-03-08 02:58:00


# Interpolating larger gaps (MICE)